In [1]:
import pickle
import pandas as pd
import numpy as np
import gc; gc.enable()
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack, load_npz

In [2]:
'''
with open('all_features.pickle', 'rb') as handle:
    all_features = pickle.load(handle)
'''      
all_features = load_npz('all_features_hv_charwb23.npz')

In [3]:
train_y = pd.read_csv("regression_target.csv")
train_y = train_y['deal_probability'].values
train_len = train_y.shape[0]
print(train_len)

1503424


In [4]:
train_features = all_features[:train_len, :]
del all_features; gc.collect()

0

In [5]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GridSearchCV, ParameterGrid, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from copy import deepcopy as cp
from wordbatch.models import FM_FTRL

In [6]:
def clip_rmse(ground_truth, predictions):
    predictions = np.clip(predictions, 0., 1.)
    return mean_squared_error(ground_truth, predictions)**.5

clip_rmse_scorer = make_scorer(clip_rmse, greater_is_better=False)

In [7]:
def simple_train_val_split_eval(default_params, X, y, params):
    train_len = int(X.shape[0]*.7)
    
    min_score = None
    best_param = None
    for param in list(ParameterGrid(params)):
        use_params = cp(default_params)
        use_params.update(param)
        print('Fitting params:\n', use_params)
        md = FM_FTRL(**use_params)
        md.fit(X[:train_len, :], y[:train_len])
        score = clip_rmse(y[train_len:], md.predict(X[train_len:]))
        print(param, score)
        
        if min_score is None or score < min_score:
            best_param = param
            min_score = score
            
    print('Best param:', best_param, '\nscore:', min_score)

In [8]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .005,
    'L1': 0.0001,
    'L2': 0.1,
    'D': train_features.shape[1],
    'D_fm': 300,
    'iters': 40,
    'seed': 719,
    'threads': 4,
    'verbose': 1
}

try_params = {
    'L1': [0.0001],
    'L2': [0.001]
}
simple_train_val_split_eval(fmftrl_default_params, train_features, train_y, try_params)

Fitting params:
 {'alpha': 0.01, 'beta': 0.005, 'L1': 0.0001, 'L2': 0.001, 'D': 357887, 'D_fm': 300, 'iters': 40, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 160367.7203989733
Total e: 149513.30132924824
Total e: 146352.09932865234
Total e: 144108.86090210264
Total e: 142308.01020120425
Total e: 140686.4994502756
Total e: 139295.8849790965
Total e: 137985.22276299744
Total e: 136823.1545105725
Total e: 135675.98044813066
Total e: 134611.246783402
Total e: 133581.08454985142
Total e: 132622.72744485753
Total e: 131694.65980167914
Total e: 130811.31240909772
Total e: 129931.30546383241
Total e: 129080.37709352793
Total e: 128295.12955093759
Total e: 127513.80439120728
Total e: 126764.54521081621
Total e: 126025.54770431136
Total e: 125332.45696620546
Total e: 124609.53100690623
Total e: 123946.98753996103
Total e: 123281.4341262151
Total e: 122655.1226073082
Total e: 122029.15481062476
Total e: 121422.83330097856
Total e: 120844.86721528867
Total e: 120259.81870092954
Total e: 1197